In [ ]:
import torch
import os
import math
from pandas import read_csv
from torchvision.io import read_image, ImageReadMode

class Datasets(torch.utils.data.Dataset):
  """
  Custom dataset class for YOLOv1 model.

  This dataset class is designed to load images and corresponding labels for training YOLOv1 model.
  It reads image paths and label files from CSV file and loads images and labels accordingly.
  The labels are expected to be in the form of bounding boxes represented by their center coordinates (x, y) and radius (r).

  Args:
      csv_file (str): Path to the CSV file containing image paths and label file names.
      img_dir (str): Directory containing the images.
      label_dir (str): Directory containing the label files.
      transform (callable, optional): Optional transform to be applied to the images. Default is None.
      S (int, optional): Size of the grid. Default is 7.
      C (int, optional): Number of classes. Default is 1.
      B (int, optional): Number of bounding boxes per grid cell. Default is 1.

  Attributes:
      annotations (DataFrame): DataFrame containing image paths and label file names.
      img_dir (str): Directory containing the images.
      label_dir (str): Directory containing the label files.
      transform (callable): Transform to be applied to the images.
      S (int): Size of the grid.
      C (int): Number of classes.
      B (int): Number of bounding boxes per grid cell.
  """
  def __init__(
      self, csv_file, img_dir, label_dir, transform = None, S=7, C=1, B=1 #Only B == 1
  ):
    self.annotations = read_csv(csv_file)
    self.img_dir = img_dir
    self.label_dir = label_dir
    self.transform = transform
    self.S = S
    self.C = C
    self.B = B

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    label_path = os.path.join(self.label_dir, self.annotations.iloc[index, 1])
    boxes = []
    with open(label_path , 'r') as f:
      for label in f.readlines():
        class_label, x, y, r = [
            float(x) if float(x) != int(float(x)) else int(float(x))
            for x in label.replace("\n", "").split()
        ]

        boxes.append([class_label, x, y, r])
    boxes = torch.tensor(boxes)

    label_matrix = torch.zeros((self.S, self.S, self.C + 4*self.B))
    for box in boxes:
      class_label, x, y, r = box.tolist()
      class_label = int(class_label)
      # i,j represents the cell row and cell column
      i, j = math.floor(self.S * y), math.floor(self.S * x)
      x_cell, y_cell = self.S * x - j, self.S * y - i
      r_for_whole_img = r
      #Cheking if (x or y > 1) or (x or y < 0) for save in corect grid
      if x >= 1:
          j=self.S-1
          x_cell+=math.ceil((x-1)*self.S)
      elif x < 0:
          j=0
          x_cell-=math.ceil(-x*self.S)
      if y >= 1:
          i=self.S-1
          y_cell+=math.ceil((y-1)*self.S)
      elif y < 0:
          i=0
          y_cell-=math.ceil(-y*self.S)
      #[klasa, pewnosc, x, y, r] if C==0: [pewnosc, x, y, r]
      x_in_grid, y_in_grid, r_in_grid = label_matrix[i, j, self.C+1], label_matrix[i, j, self.C+2], label_matrix[i, j, self.C+3]
      if (label_matrix[i, j, self.C] == 0 or #empty grid
        (not(0 < x_in_grid < 1 or 0 < y_in_grid < 1) and (0 < x_cell < 1 and 0 < y_cell < 1)) or #new object have center in grid
        ((not(0 < x_in_grid < 1 or 0 < y_in_grid < 1) and not(0 < x_cell < 1 or 0 < y_cell < 1)) and r_in_grid < r_for_whole_img) or # new object doesnt have center in grid but older too, so take that one with bigger r
        ((0 < x_in_grid < 1 and 0 < y_in_grid < 1) and (0 < x_cell < 1 and 0 < y_cell < 1) and r_in_grid < r_for_whole_img)): #new object have center in grid older too, but new one is bigger
        label_matrix[i, j, self.C ] = 1  #<= probability = 1
        box_coordinates = torch.tensor(
                  [x_cell, y_cell, r_for_whole_img]
              )
        label_matrix[i, j , self.C+1 : self.C+4] = box_coordinates
      else:
        continue
      label_matrix[i, j, class_label] = 1

    img_path = os.path.join(self.img_dir, self.annotations.iloc[index, 0])
    image = read_image(img_path, mode = ImageReadMode.RGB)
    if self.transform:
      image = self.transform(image)

    return image, label_matrix